In [1]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv('/Users/winirrr/Documents/EventBased_Project/exp_data/exp_0_df.csv')

def normalize_events(timestamps, x, y, polarity, resolution):
    sensor_height, sensor_width = resolution

    min_t, max_t = timestamps.min(), timestamps.max()
    timestamps = (timestamps - min_t) / (max_t - min_t)  # Normalize to [0, 1]

    x = x / sensor_width
    y = y / sensor_height

    return timestamps, x, y, polarity

# Extract relevant information
timestamps = df['timestamp'].values
x = df['x'].values
y = df['y'].values
polarity = df['polarity'].values
labels = df['labels'].values

# Normalize the events
resolution = (640, 480)  # Example resolution, adjust as needed
timestamps, x, y, polarity = normalize_events(timestamps, x, y, polarity, resolution)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class LIFNeuronLayer(nn.Module):
    def __init__(self, num_neurons, tau_m=20.0, V_rest=-65.0, V_thresh=-50.0, V_reset=-70.0, R=1.0, dt=1.0):
        super(LIFNeuronLayer, self).__init__()
        self.num_neurons = num_neurons
        self.tau_m = tau_m
        self.V_rest = V_rest
        self.V_thresh = V_thresh
        self.V_reset = V_reset
        self.R = R
        self.dt = dt
        self.V = None
        self.spike = None
    
    def forward(self, input_current):
        if self.V is None or self.V.size(0) != input_current.size(0):
            batch_size = input_current.size(0)
            self.V = torch.ones(batch_size, self.num_neurons) * self.V_rest
            self.spike = torch.zeros(batch_size, self.num_neurons)
        
        dV = (-(self.V - self.V_rest) + self.R * input_current) * (self.dt / self.tau_m)
        self.V += dV
        self.spike = (self.V >= self.V_thresh).float()
        self.V = torch.where(self.V >= self.V_thresh, torch.ones_like(self.V) * self.V_reset, self.V)
        return self.spike

class SNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.hidden = LIFNeuronLayer(hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        out = self.hidden(x)
        out = self.fc2(out)
        return out

# Define hyperparameters
input_size = 4  # Number of features (timestamp, x, y, polarity)
hidden_size = 100  # Number of neurons in the hidden layer
output_size = len(set(labels))  # Number of classes

# Create SNN model
snn = SNN(input_size, hidden_size, output_size)


In [3]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Prepare data for PyTorch
X = np.vstack([timestamps, x, y, polarity]).T
y = np.array([0 if label == 'press' else 1 for label in labels])  # Example binary labels, adjust as needed

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(snn.parameters(), lr=0.001)

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    snn.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = snn(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    # Print training progress
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 0.6897
Epoch [2/100], Loss: 0.6939
Epoch [3/100], Loss: 0.6942
Epoch [4/100], Loss: 0.6916
Epoch [5/100], Loss: 0.6929
Epoch [6/100], Loss: 0.6924
Epoch [7/100], Loss: 0.6934
Epoch [8/100], Loss: 0.6929
Epoch [9/100], Loss: 0.6934
Epoch [10/100], Loss: 0.6931
Epoch [11/100], Loss: 0.6931
Epoch [12/100], Loss: 0.6931
Epoch [13/100], Loss: 0.6934
Epoch [14/100], Loss: 0.6933
Epoch [15/100], Loss: 0.6941
Epoch [16/100], Loss: 0.6945
Epoch [17/100], Loss: 0.6929
Epoch [18/100], Loss: 0.6931
Epoch [19/100], Loss: 0.6929
Epoch [20/100], Loss: 0.6935
Epoch [21/100], Loss: 0.6935
Epoch [22/100], Loss: 0.6932
Epoch [23/100], Loss: 0.6932
Epoch [24/100], Loss: 0.6935
Epoch [25/100], Loss: 0.6932
Epoch [26/100], Loss: 0.6927
Epoch [27/100], Loss: 0.6936
Epoch [28/100], Loss: 0.6935
Epoch [29/100], Loss: 0.6946
Epoch [30/100], Loss: 0.6932
Epoch [31/100], Loss: 0.6927
Epoch [32/100], Loss: 0.6924
Epoch [33/100], Loss: 0.6942
Epoch [34/100], Loss: 0.6932
Epoch [35/100], Loss: 0

In [4]:
snn.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = snn(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 48.00%
